# Generate statistics table (Table E2)


In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
%load_ext autoreload
%autoreload 2

<IPython.core.display.Javascript object>

In [4]:
import datetime as dt
import logging
import os.path
from glob import glob
from tqdm.auto import tqdm

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scmdata
import scmdata.database
import seaborn as sns

from ndcs.constants import PROCESSED_DATA_DIR, NDC_TARGET_RELEASE

<IPython.core.display.Javascript object>

In [5]:
plt.rcParams["figure.figsize"] = (12, 8)

<IPython.core.display.Javascript object>

In [6]:
ZENODO_VERSION = "v1.0.0"

<IPython.core.display.Javascript object>

In [7]:
ZENODO_DATA_DIR = os.path.join(PROCESSED_DATA_DIR, "zenodo", ZENODO_VERSION)

os.makedirs(ZENODO_DATA_DIR, exist_ok=True)

<IPython.core.display.Javascript object>

# Fetch data from AWS

In [7]:
CLIMATE_ASSESSMENT_ID = "20211125-ndc-sensitivity"
DATA_DIR_SENSITIVITY = os.path.join(
    PROCESSED_DATA_DIR, "climate_assessment", CLIMATE_ASSESSMENT_ID
)

!aws s3 sync s3://ipcc-climate-assessment/{CLIMATE_ASSESSMENT_ID}/out/raw_climate_output/ {DATA_DIR_SENSITIVITY}/raw_climate_output/

meta_raw_sensitivity = pd.read_csv(
    glob(os.path.join(DATA_DIR_SENSITIVITY, "results_*.csv"))[0]
)

<IPython.core.display.Javascript object>

In [8]:
CLIMATE_ASSESSMENT_ID = "20211125-ndc-methanePledges"
DATA_DIR_METHANE = os.path.join(
    PROCESSED_DATA_DIR, "climate_assessment", CLIMATE_ASSESSMENT_ID
)

!aws s3 sync s3://ipcc-climate-assessment/{CLIMATE_ASSESSMENT_ID}/out/raw_climate_output/ {DATA_DIR_METHANE}/raw_climate_output/
!aws s3 sync s3://ipcc-climate-assessment/{CLIMATE_ASSESSMENT_ID}/merged/ {DATA_DIR_METHANE}/

meta_raw_methane = pd.read_csv(glob(os.path.join(DATA_DIR_METHANE, "results_*.csv"))[0])

<IPython.core.display.Javascript object>

## Load

In [9]:
meta_raw = pd.concat([meta_raw_sensitivity, meta_raw_methane]).reset_index(drop=True)
meta_raw.head()

,model,scenario,exclude,Exceedance Probability 1.5C (MAGICCv7.5.3),Exceedance Probability 2.0C (MAGICCv7.5.3),Exceedance Probability 2.5C (MAGICCv7.5.3),Exceedance Probability 3.0C (MAGICCv7.5.3),p33 peak warming (MAGICCv7.5.3),median peak warming (MAGICCv7.5.3),p67 peak warming (MAGICCv7.5.3),...,p67 year of peak warming (MAGICCv7.5.3),p33 warming in 2100 (MAGICCv7.5.3),median warming in 2100 (MAGICCv7.5.3),p67 warming in 2100 (MAGICCv7.5.3),Category,Category_name,harmonization,infilling,climate-models,pipeline
0,NDC Factsheet,2021-11-09_1__high__U__rate__exclude,False,0.920000,0.468333,0.136667,0.031667,1.808845,1.972420,2.147504,...,2094,1.765209,1.948868,2.136743,C4,C4: Below 2°C,aneris (version: 0.2.0+41.g6ddf6e5),silicone (version: 1.2.1),openscm_runner (version: 0.9.1),climate-assessment (version: 0.1.3-fgd+24.g0f1...
1,NDC Factsheet,2021-11-09_1__high__U__rate__include,False,0.980000,0.731667,0.333333,0.130000,2.087909,2.285953,2.504518,...,2100,2.078915,2.276149,2.501708,C5,C5: Below 2.5°C,aneris (version: 0.2.0+41.g6ddf6e5),silicone (version: 1.2.1),openscm_runner (version: 0.9.1),climate-assessment (version: 0.1.3-fgd+24.g0f1...
2,NDC Factsheet,2021-11-09_1__high__U__2030__exclude,False,0.991667,0.806667,0.445000,0.168333,2.212413,2.420234,2.655731,...,2100,2.210545,2.419308,2.654907,C5,C5: Below 2.5°C,aneris (version: 0.2.0+41.g6ddf6e5),silicone (version: 1.2.1),openscm_runner (version: 0.9.1),climate-assessment (version: 0.1.3-fgd+24.g0f1...
3,NDC Factsheet,2021-11-09_1__high__U__2030__include,False,1.000000,0.965000,0.755000,0.420000,2.635075,2.893571,3.136713,...,2100,2.635075,2.893571,3.136713,C6,C6: Below 3.0°C,aneris (version: 0.2.0+41.g6ddf6e5),silicone (version: 1.2.1),openscm_runner (version: 0.9.1),climate-assessment (version: 0.1.3-fgd+24.g0f1...
4,NDC Factsheet,2021-11-09_1__low__C__constant__exclude,False,0.895000,0.398333,0.110000,0.015000,1.746747,1.899191,2.058014,...,2070,1.661435,1.837565,2.020847,C4,C4: Below 2°C,aneris (version: 0.2.0+41.g6ddf6e5),silicone (version: 1.2.1),openscm_runner (version: 0.9.1),climate-assessment (version: 0.1.3-fgd+24.g0f1...


<IPython.core.display.Javascript object>

In [10]:
# drop peak warming and year of peak warming stats
cols = [
    c
    for c in meta_raw.columns
    if ("peak warming" not in c and "warming in 2100" not in c)
]
cols
meta = meta_raw[cols]
cols = [c.replace("(MAGICCv7.5.3)", "") for c in cols]
meta.columns = cols
meta.head()

,model,scenario,exclude,Exceedance Probability 1.5C,Exceedance Probability 2.0C,Exceedance Probability 2.5C,Exceedance Probability 3.0C,Category,Category_name,harmonization,infilling,climate-models,pipeline
0,NDC Factsheet,2021-11-09_1__high__U__rate__exclude,False,0.920000,0.468333,0.136667,0.031667,C4,C4: Below 2°C,aneris (version: 0.2.0+41.g6ddf6e5),silicone (version: 1.2.1),openscm_runner (version: 0.9.1),climate-assessment (version: 0.1.3-fgd+24.g0f1...
1,NDC Factsheet,2021-11-09_1__high__U__rate__include,False,0.980000,0.731667,0.333333,0.130000,C5,C5: Below 2.5°C,aneris (version: 0.2.0+41.g6ddf6e5),silicone (version: 1.2.1),openscm_runner (version: 0.9.1),climate-assessment (version: 0.1.3-fgd+24.g0f1...
2,NDC Factsheet,2021-11-09_1__high__U__2030__exclude,False,0.991667,0.806667,0.445000,0.168333,C5,C5: Below 2.5°C,aneris (version: 0.2.0+41.g6ddf6e5),silicone (version: 1.2.1),openscm_runner (version: 0.9.1),climate-assessment (version: 0.1.3-fgd+24.g0f1...
3,NDC Factsheet,2021-11-09_1__high__U__2030__include,False,1.000000,0.965000,0.755000,0.420000,C6,C6: Below 3.0°C,aneris (version: 0.2.0+41.g6ddf6e5),silicone (version: 1.2.1),openscm_runner (version: 0.9.1),climate-assessment (version: 0.1.3-fgd+24.g0f1...
4,NDC Factsheet,2021-11-09_1__low__C__constant__exclude,False,0.895000,0.398333,0.110000,0.015000,C4,C4: Below 2°C,aneris (version: 0.2.0+41.g6ddf6e5),silicone (version: 1.2.1),openscm_runner (version: 0.9.1),climate-assessment (version: 0.1.3-fgd+24.g0f1...


<IPython.core.display.Javascript object>

In [11]:
db_sensitivity = scmdata.database.ScmDatabase(
    os.path.join(DATA_DIR_SENSITIVITY, "raw_climate_output"),
    ["climate_model", "variable", "model", "scenario"],
)
db_sensitivity

<scmdata.database.SCMDatabase (root_dir: /data2/ubuntu-mmeinshausen/unfccc-ndc/src/../data/processed/climate_assessment/20211125-ndc-sensitivity/raw_climate_output, levels: ('climate_model', 'variable', 'model', 'scenario'))>

<IPython.core.display.Javascript object>

In [12]:
db_methane = scmdata.database.ScmDatabase(
    os.path.join(DATA_DIR_METHANE, "raw_climate_output"),
    ["climate_model", "variable", "model", "scenario"],
)
db_methane

<scmdata.database.SCMDatabase (root_dir: /data2/ubuntu-mmeinshausen/unfccc-ndc/src/../data/processed/climate_assessment/20211125-ndc-methanePledges/raw_climate_output, levels: ('climate_model', 'variable', 'model', 'scenario'))>

<IPython.core.display.Javascript object>

In [13]:
temperatures_raw = scmdata.run_append(
    [
        db_sensitivity.load(variable="Raw Surface Temperature (GSAT)"),
        db_methane.load(variable="Raw Surface Temperature (GSAT)"),
    ]
)


HIST_TEMP_REF_PERIOD = range(1850, 1900 + 1)
HIST_TEMP_EVALUATION_PERIOD = range(1995, 2014 + 1)
HIST_TEMP_ASSESSMENT_CENTRAL = 0.85

temperatures = temperatures_raw.relative_to_ref_period_mean(
    year=HIST_TEMP_REF_PERIOD
).drop_meta(["reference_period_start_year", "reference_period_end_year"])
temperatures = temperatures.adjust_median_to_target(
    HIST_TEMP_ASSESSMENT_CENTRAL,
    HIST_TEMP_EVALUATION_PERIOD,
    process_over=("run_id",),
).filter(year=range(2000, 2101))

Loading files:   0%|          | 0/32 [00:00<?, ?it/s]

Loading files:   0%|          | 0/3 [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

In [14]:
percentiles = (
    (0.05, "p5"),
    (0.37, "p37"),
    (0.50, "median"),
    (0.067, "p67"),
    (0.95, "p95"),
)

<IPython.core.display.Javascript object>

In [15]:
stats = []

for r in temperatures.groupby("scenario"):
    scenario_stats = {
        "scenario": r.get_unique_meta("scenario", True),
    }

    for p, p_long in percentiles:
        scenario_stats["{} peak warming".format(p_long)] = (
            r.timeseries().max(axis=1).quantile(p)
        )

    for p, p_long in percentiles:
        scenario_stats["{} year of peak warming".format(p_long)] = (
            r.timeseries().idxmax(axis=1).quantile(p).year
        )

    for p, p_long in percentiles:
        scenario_stats["{} warming in 2100".format(p_long)] = (
            r.filter(year=2100).timeseries().squeeze().quantile(p)
        )
    stats.append(scenario_stats)
stats = pd.DataFrame(stats).set_index("scenario")

<IPython.core.display.Javascript object>

In [16]:
merged = pd.merge(meta, stats, left_on="scenario", right_on="scenario")

<IPython.core.display.Javascript object>

In [17]:
merged_cols = [
    "model",
    "scenario",
    "Category",
    "Category_name",
    "Exceedance Probability 1.5C ",
    "Exceedance Probability 2.0C ",
    "Exceedance Probability 2.5C ",
    "Exceedance Probability 3.0C ",
    "p5 peak warming",
    "p37 peak warming",
    "median peak warming",
    "p67 peak warming",
    "p95 peak warming",
    "p5 year of peak warming",
    "p37 year of peak warming",
    "median year of peak warming",
    "p67 year of peak warming",
    "p95 year of peak warming",
    "p5 warming in 2100",
    "p37 warming in 2100",
    "median warming in 2100",
    "p67 warming in 2100",
    "p95 warming in 2100",
    "harmonization",
    "infilling",
    "climate-models",
    "pipeline",
]
merged = merged[merged_cols]
merged

,model,scenario,Category,Category_name,Exceedance Probability 1.5C,Exceedance Probability 2.0C,Exceedance Probability 2.5C,Exceedance Probability 3.0C,p5 peak warming,p37 peak warming,...,p95 year of peak warming,p5 warming in 2100,p37 warming in 2100,median warming in 2100,p67 warming in 2100,p95 warming in 2100,harmonization,infilling,climate-models,pipeline
0,NDC Factsheet,2021-11-09_1__high__U__rate__exclude,C4,C4: Below 2°C,0.920000,0.468333,0.136667,0.031667,1.442840,1.838510,...,2100,1.366835,1.801970,1.948868,1.398653,2.902575,aneris (version: 0.2.0+41.g6ddf6e5),silicone (version: 1.2.1),openscm_runner (version: 0.9.1),climate-assessment (version: 0.1.3-fgd+24.g0f1...
1,NDC Factsheet,2021-11-09_1__high__U__rate__include,C5,C5: Below 2.5°C,0.980000,0.731667,0.333333,0.130000,1.639121,2.123392,...,2100,1.614208,2.116397,2.276149,1.652774,3.397142,aneris (version: 0.2.0+41.g6ddf6e5),silicone (version: 1.2.1),openscm_runner (version: 0.9.1),climate-assessment (version: 0.1.3-fgd+24.g0f1...
2,NDC Factsheet,2021-11-09_1__high__U__2030__exclude,C5,C5: Below 2.5°C,0.991667,0.806667,0.445000,0.168333,1.727773,2.250820,...,2100,1.717917,2.250820,2.419308,1.754845,3.603234,aneris (version: 0.2.0+41.g6ddf6e5),silicone (version: 1.2.1),openscm_runner (version: 0.9.1),climate-assessment (version: 0.1.3-fgd+24.g0f1...
3,NDC Factsheet,2021-11-09_1__high__U__2030__include,C6,C6: Below 3.0°C,1.000000,0.965000,0.755000,0.420000,2.070238,2.685196,...,2100,2.070238,2.685196,2.893571,2.131043,4.242162,aneris (version: 0.2.0+41.g6ddf6e5),silicone (version: 1.2.1),openscm_runner (version: 0.9.1),climate-assessment (version: 0.1.3-fgd+24.g0f1...
4,NDC Factsheet,2021-11-09_1__low__C__constant__exclude,C4,C4: Below 2°C,0.895000,0.398333,0.110000,0.015000,1.403575,1.771608,...,2100,1.293084,1.696339,1.837565,1.317919,2.743235,aneris (version: 0.2.0+41.g6ddf6e5),silicone (version: 1.2.1),openscm_runner (version: 0.9.1),climate-assessment (version: 0.1.3-fgd+24.g0f1...
5,NDC Factsheet,2021-11-09_1__low__C__constant__include,C4,C4: Below 2°C,0.906667,0.425000,0.123333,0.018333,1.420002,1.795033,...,2100,1.314433,1.726375,1.869807,1.340433,2.792324,aneris (version: 0.2.0+41.g6ddf6e5),silicone (version: 1.2.1),openscm_runner (version: 0.9.1),climate-assessment (version: 0.1.3-fgd+24.g0f1...
6,NDC Factsheet,2021-11-09_1__high__C__SSP1BL__exclude,C4,C4: Below 2°C,0.890000,0.375000,0.105000,0.015000,1.392404,1.758191,...,2100,1.282088,1.681064,1.819243,1.306402,2.717334,aneris (version: 0.2.0+41.g6ddf6e5),silicone (version: 1.2.1),openscm_runner (version: 0.9.1),climate-assessment (version: 0.1.3-fgd+24.g0f1...
7,NDC Factsheet,2021-11-09_1__high__C__SSP1BL__include,C4,C4: Below 2°C,0.900000,0.411667,0.118333,0.018333,1.409682,1.784912,...,2100,1.307429,1.717240,1.858914,1.333154,2.775235,aneris (version: 0.2.0+41.g6ddf6e5),silicone (version: 1.2.1),openscm_runner (version: 0.9.1),climate-assessment (version: 0.1.3-fgd+24.g0f1...
8,NDC Factsheet,2021-11-09_1__low__U__constant__exclude,C4,C4: Below 2°C,0.916667,0.436667,0.125000,0.025000,1.428772,1.805061,...,2100,1.321703,1.736075,1.880146,1.348062,2.809986,aneris (version: 0.2.0+41.g6ddf6e5),silicone (version: 1.2.1),openscm_runner (version: 0.9.1),climate-assessment (version: 0.1.3-fgd+24.g0f1...
9,NDC Factsheet,2021-11-09_1__low__U__constant__include,C4,C4: Below 2°C,0.925000,0.480000,0.136667,0.030000,1.453962,1.846663,...,2100,1.364555,1.797325,1.944548,1.396065,2.902544,aneris (version: 0.2.0+41.g6ddf6e5),silicone (version: 1.2.1),openscm_runner (version: 0.9.1),climate-assessment (version: 0.1.3-fgd+24.g0f1...


<IPython.core.display.Javascript object>

In [19]:
merged.round(3).to_csv(
    os.path.join(ZENODO_DATA_DIR, "stats_{}.csv".format(NDC_TARGET_RELEASE)),
    index=False,
)

<IPython.core.display.Javascript object>